In [ ]:
import pandas as pd 
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [ ]:
# 접속 주소
url = "https://comp.wisereport.co.kr/company/c1010001.aspx?cmp_cd="
# 종목의 코드 (하이닉스 종목코드)
input_code = "000660"

### 연습 
- webdriver의 Chrome을 이용하여 url + input_code 로 요청 
- driver의 html 문서를 불러온다. 
- 불러온 html을 BeautifulSoup을 이용하여 데이터 파싱 
- div 태그 중 class의 이름이 "cmp_comment"와 같은 태그를 선택 
    - li 태그들을 모두 찾아서 텍스트 추출하여 새로운 리스트를 생성(cmp_info)
- div 태그 중 class의 이름이 "cmp_comment_etc"와 같은 태그를 선택 
    - li 태그들을 모두 찾아서 텍스트 추출하여 새로운 리스트를 생성(cmp_etc)
- DataFrame 생성하여 cmp_info, cmp_etc 을 대입 

In [ ]:
driver = webdriver.Chrome()

In [ ]:
# url + input_code를 이용해서 요청 
driver.get(url+input_code)

In [ ]:
soup = bs(
    driver.page_source, 'html.parser'
)

In [ ]:
driver.quit()

In [ ]:
# div 태그중 class가 'cmp_comment'인 태그를 선택 
div_data = soup.find(
    'div', attrs={
        'class' : 'cmp_comment'
    }
)
div_data

In [ ]:
# div_data에서 li 태그들을 모두 찾는다. 
li_list = div_data.find_all('li')

In [ ]:
cmp_info = []
for li_data in li_list:
    # li_data는 Tag data 대입 
    cmp_info.append(li_data.get_text())
cmp_info

In [ ]:
cmp_etc = list(
    map(
        lambda x : x.get_text(),
        soup.find('div', 
            attrs={'class' : 'cmp_comment_etc'}
        ).find_all('li')
    )
)

In [ ]:
df1 = pd.DataFrame(
    [
        cmp_info, cmp_etc
    ], index = ['cmp_info', 'cmp_etc']
)
df1

In [ ]:
df2 = pd.DataFrame(
    {
        'cmp_info' : cmp_info, 
        'cmp_etc' : cmp_etc
    }
)
df2

In [32]:
# df1의 인덱스를 컬럼으로 컬럼을 인덱스로 변환하는 전치 
df2['code'] = input_code

In [33]:
df2

,cmp_info,cmp_etc,code
0,동사는 1949년 설립되어 경기도 이천시에 본사를 두고 4개의 생산기지와 3개의 연...,"2025년 상반기 전년동기 대비 연결기준 매출액은 38.2% 증가, 영업이익은 99...",000660
1,"메모리 반도체와 Foundry 사업을 영위하며, DRAM과 NAND Flash를 주...",AI향 메모리 수요 성장과 우호적 가격 환경으로 DRAM과 NAND 출하량이 증가했...,000660
2,생성형 AI 시장의 신기술 등장과 시장 영향력 확대에 따라 고성능/고용량 메모리 솔...,"AI 서버와 데이터센터 확산으로 HBM 수요 급증, DRAM은 프리미엄 제품 성장이...",000660


In [34]:
# 데이터프레임의 행 결합 
    # 조건 -> 데이터프레임의 구조가 같다.
# pandas 안에 존재하는 concat() 함수를 이용
# concat() -> 단순한 데이터프레임의 결합 
    # 첫번째 인자 -> 데이터프레임들 [df, df2, df3, ...]
    # axis 매개변수 -> 행으로 결합할것인가, 열로 결합 할것인가?
    # ingore_index -> 결합 후 인덱스를 초기화할것인가?

In [35]:
import requests

In [36]:
input_code = "005180"

In [37]:
res = requests.get(url+input_code)

In [38]:
res

<Response [200]>

In [39]:
soup = bs(res.text, 'html.parser')

In [ ]:
cmp_info = list(
    map(
        lambda x : x.get_text(),
        soup.find('div', attrs={'class' : 'cmp_comment'}).find_all('li')
    )
)
cmp_etc = list(
    map(
        lambda x : x.get_text(), 
        soup.find('div', attrs={'class' : 'cmp_comment_etc'}).find_all('li')
    )
)

In [41]:
df3 = pd.DataFrame({'cmp_info' : cmp_info, 'cmp_etc' : cmp_etc})
df3['code'] = input_code

In [ ]:
pd.concat([df2, df3], axis = 0, ignore_index=True)

,cmp_info,cmp_etc,code
0,동사는 1949년 설립되어 경기도 이천시에 본사를 두고 4개의 생산기지와 3개의 연...,"2025년 상반기 전년동기 대비 연결기준 매출액은 38.2% 증가, 영업이익은 99...",000660
1,"메모리 반도체와 Foundry 사업을 영위하며, DRAM과 NAND Flash를 주...",AI향 메모리 수요 성장과 우호적 가격 환경으로 DRAM과 NAND 출하량이 증가했...,000660
2,생성형 AI 시장의 신기술 등장과 시장 영향력 확대에 따라 고성능/고용량 메모리 솔...,"AI 서버와 데이터센터 확산으로 HBM 수요 급증, DRAM은 프리미엄 제품 성장이...",000660
3,동사는 1967년 설립된 유가증권시장 상장 유가공 전문기업임.,"2025년 상반기 전년동기 대비 연결기준 매출액은 1.4% 증가, 영업이익은 38....",005180
4,"바나나맛우유, 요플레, 투게더, 아카페라, 따옴 등 주요 제품을 보유하고 있으며, ...","유가공 사업은 원재료 가격 상승과 일회성 비용으로 수익성이 하락했으나, 고품질 정책...",005180
5,"남양주, 광주, 김해, 논산, 경산에 생산공장을 보유하고 있으며, 식품연구소 내 6...","국내는 기능성 제품 수요가 확산되고, 해외는 중국의 고품질 선호도, 베트남의 한류 ...",005180
